# Migrating to Common Backtester

Goal: show sames results but using this common backtester

In [ ]:
""" 
Imports
"""

import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import coint
from itertools import combinations
from tqdm import tqdm

In [ ]:
# Get data
df = pd.read_csv('./crypto_data.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

,ARKUSDT_open,ARKUSDT_high,ARKUSDT_low,ARKUSDT_close,ARKUSDT_volume,ARKUSDT_turnover,AXLUSDT_open,AXLUSDT_high,AXLUSDT_low,AXLUSDT_close,...,ZRXUSDT_low,ZRXUSDT_close,ZRXUSDT_volume,ZRXUSDT_turnover,GRTUSDT_open,GRTUSDT_high,GRTUSDT_low,GRTUSDT_close,GRTUSDT_volume,GRTUSDT_turnover
time,,,,,,,,,,,,,,,,,,,,,
2024-06-01 00:00:00,0.8052,0.8294,0.8031,0.8268,576568.0,470695.6409,1.0340,1.0613,1.0318,1.0531,...,0.5190,0.5205,58775.0,30580.0258,0.29844,0.29984,0.29794,0.29920,230545.7,68941.066258
2024-06-01 01:00:00,0.8268,0.8437,0.8127,0.8153,1026640.0,852475.3372,1.0531,1.0654,1.0429,1.0586,...,0.5192,0.5195,25026.0,13013.3420,0.29920,0.29975,0.29857,0.29926,223235.0,66801.134862
2024-06-01 02:00:00,0.8153,0.8325,0.8124,0.8266,350101.0,287436.0344,1.0586,1.0653,1.0527,1.0611,...,0.5186,0.5199,63868.0,33181.7229,0.29926,0.29959,0.29860,0.29929,161906.8,48417.999107
2024-06-01 03:00:00,0.8266,0.8318,0.8220,0.8247,391127.0,323734.8000,1.0611,1.0699,1.0496,1.0508,...,0.5171,0.5174,96745.0,50157.0848,0.29929,0.29968,0.29788,0.29795,447063.9,133582.787508
2024-06-01 04:00:00,0.8247,0.8257,0.8184,0.8235,128711.0,105733.7135,1.0508,1.0529,1.0405,1.0458,...,0.5173,0.5214,95808.0,49866.4207,0.29795,0.29931,0.29761,0.29906,195045.3,58186.611414


In [ ]:
""" 
Backtest DF:

1. Contains price for each item with X_price, and BTCUSDT_price
2. Contains position for each item at each time iterval with X_pos
    note that pos is in $, and must sum to 1
"""
# Define date ranges
start_time = datetime(2024, 6, 1, 0, 0)
end_time = datetime(2024, 8, 25, 0, 0)
test_end_time = datetime(2024, 10, 7, 0, 0)
increment = timedelta(hours=1)

# Subset of df
df_timeframe = df.loc[start_time:end_time, :]

def test_statarb(stock1, stock2, slope, intercept, signalStd=1):
    col1 = f'{stock1}_close'
    col2 = f'{stock2}_close'

    # Add signal
    signal = df_timeframe[col1] - (slope*df_timeframe[col2] + intercept)
    signal = signal / signalStd # signal normalized
    df_timeframe['signal'] = signal


    position_dollars = abs(df_timeframe[col1]) + abs(df_timeframe[col2]*slope) # per 1 signal, this is dollars
    position_volume = signal * (0.1/position_dollars) # 1 std signal --> $0.1 worth of stock

    desired_position_1 = -position_volume*df_timeframe[col1] # scale by price as well
    desired_position_2 = slope*position_volume*df_timeframe[col2] # scale by price

    # Add relevant cols
    final_df = pd.DataFrame()
    final_df[f'{stock1}_price'] = df_timeframe[col1]
    final_df[f'{stock2}_price'] = df_timeframe[col2]
    final_df[f'{stock1}_pos'] = desired_position_1
    final_df[f'{stock2}_pos'] = desired_position_2
    final_df['BTCUSDT_price'] = df_timeframe['BTCUSDT_close']
    return final_df

res = test_statarb('BOBAUSDT', 'OPPERP', 0.11161, 0.055405, 0.0069314)
res.head()

/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_55210/3614602828.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timeframe['signal'] = signal


,BOBAUSDT_price,OPPERP_price,BOBAUSDT_pos,OPPERP_pos,BTCUSDT_price
time,,,,,
2024-06-01 00:00:00,0.34622,2.4805,-0.111964,0.089530,67704.7
2024-06-01 01:00:00,0.34540,2.4750,-0.110304,0.088216,67721.9
2024-06-01 02:00:00,0.34456,2.4745,-0.103915,0.083292,67725.3
2024-06-01 03:00:00,0.34453,2.4675,-0.110065,0.087980,67656.0
2024-06-01 04:00:00,0.34777,2.4785,-0.126475,0.100602,67744.8


In [ ]:
# Try backtest
import sys
sys.path.append('../')  # Go up one directory level
from backtester import BackTester

b = BackTester(res)
b.plot_individual_pnl()